In [1]:
import xml.etree.ElementTree as ET
import os

In [5]:
#See sample xml file for reference
def parse_annotation(ann_dir, img_dir, labels=[], x_ratio =1, y_ratio = 1):
    all_imgs = []
    seen_labels = {}
    
    for ann in sorted(os.listdir(ann_dir)):
        img = {'object':[]}

        tree = ET.parse(os.path.join(ann_dir, ann))
        
        for elem in tree.iter():
            if 'filename' in elem.tag:
                img['filename'] = os.path.join(img_dir, elem.text)
            if 'width' in elem.tag:
                img['width'] = int(elem.text)
            if 'height' in elem.tag:
                img['height'] = int(elem.text)
            if 'object' in elem.tag or 'part' in elem.tag:
                obj = {}
                
                for attr in list(elem):
                    if 'name' in attr.tag:
                        obj['name'] = attr.text

                        if obj['name'] in seen_labels:
                            seen_labels[obj['name']] += 1
                        else:
                            seen_labels[obj['name']] = 1
                        
                        if len(labels) > 0 and obj['name'] not in labels:
                            break
                        else:
                            img['object'] += [obj]
                            
                    if 'bndbox' in attr.tag:
                        for dim in list(attr):
                            if 'xmin' in dim.tag:
                                obj['xmin'] = int(round(float(dim.text))*x_ratio)
                            if 'ymin' in dim.tag:
                                obj['ymin'] = int(round(float(dim.text))*y_ratio)
                            if 'xmax' in dim.tag:
                                obj['xmax'] = int(round(float(dim.text))*x_ratio)
                            if 'ymax' in dim.tag:
                                obj['ymax'] = int(round(float(dim.text))*y_ratio)

        if len(img['object']) > 0:
            all_imgs += [img]
                        
    return all_imgs, seen_labels

In [6]:
LABELS = ['garbage'] # array containing labels. Can be more than one.
train_image_folder = '/mnt/DABA7127BA7100F5/projects/data/pi_data'
train_annot_folder = '/mnt/DABA7127BA7100F5/projects/data/pi_labels'

train_imgs, seen_train_labels = parse_annotation(train_annot_folder, train_image_folder, labels=LABELS,)

#label_ids = seen_train_labels.copy()
labels_ids = {'garbage': 0}

In [7]:
train_imgs #Checking data format

[{'object': [{'name': 'garbage',
    'xmin': 362,
    'ymin': 60,
    'xmax': 374,
    'ymax': 72},
   {'name': 'garbage', 'xmin': 227, 'ymin': 67, 'xmax': 251, 'ymax': 83},
   {'name': 'garbage', 'xmin': 478, 'ymin': 342, 'xmax': 593, 'ymax': 437}],
  'filename': '/mnt/DABA7127BA7100F5/projects/data/pi_data/pi_1.jpg',
  'width': 800,
  'height': 480},
 {'object': [{'name': 'garbage',
    'xmin': 235,
    'ymin': 266,
    'xmax': 246,
    'ymax': 277},
   {'name': 'garbage', 'xmin': 461, 'ymin': 340, 'xmax': 593, 'ymax': 435}],
  'filename': '/mnt/DABA7127BA7100F5/projects/data/pi_data/pi_10.jpg',
  'width': 800,
  'height': 480},
 {'object': [{'name': 'garbage',
    'xmin': 189,
    'ymin': 166,
    'xmax': 197,
    'ymax': 178},
   {'name': 'garbage', 'xmin': 353, 'ymin': 2, 'xmax': 364, 'ymax': 8}],
  'filename': '/mnt/DABA7127BA7100F5/projects/data/pi_data/pi_1000.jpg',
  'width': 800,
  'height': 480},
 {'object': [{'name': 'garbage',
    'xmin': 686,
    'ymin': 316,
    'xmax': 

In [8]:
f = open ('pi_labels.txt', 'w+')
for img_number in range(0,len(train_imgs)):
    image = train_imgs[img_number]
    im_name = image['filename'].split('/')[-1]
    objects = image['object']
    # print (im_name)
    line = im_name
    for objs in objects:
        xmin = objs['xmin']
        ymin = objs['ymin']
        xmax = objs['xmax']
        ymax = objs['ymax']
        c_id = labels_ids[objs['name']]
#         print (xmin, ymin, xmax, ymax, c_id)
        line += ' '+str(xmin)+',' +str(ymin)+',' +str(xmax)+','+str(ymax)+','+ str(c_id)

#     print (line)
    f.write (line + '\n')
f.close()